In [89]:
import os
import random

from pytorch3d.io import load_obj
from pytorch3d.ops import sample_farthest_points

import numpy as np
import torch

In [86]:
SIZEKOREA_INFERENCE =  '/workspace/SilhouettesbasedHSE/output/sizekorea'
SIZEKOREA_RVH = '/workspace/datasets/sizekorea/rvh_test'

OUTPUT_PATH = '/workspace/SilhouettesbasedHSE/merge_rvh_inference'

In [87]:
LABELING_CSV_PATH = '/workspace/RVH_Mesh_Registration/RVH_result_labeling.csv'

def get_label_names(csv_path):
    labeled_names = set()

    with open(csv_path, 'rt') as f:
        for l in f.readlines():
            l = l.strip()
            rvh_name, label, _ = l.split(',')

            if label == '1':
                labeled_names.add(rvh_name)

    return labeled_names

names = list(get_label_names(LABELING_CSV_PATH))

In [88]:
random.seed(42)
random.shuffle(names)

for n in names[:300]: 
    rvh = f'/workspace/datasets/sizekorea/rvh_test/{n}/{n}E_m.obj'
    merge = f'/workspace/SilhouettesbasedHSE/output/sizekorea/{n}/merged.obj'
    v_rvh, _, _ = load_obj(rvh)
    
    v_rvh, _ = sample_farthest_points(v_rvh.unsqueeze(0), K=6890)
    v_rvh = v_rvh[0]

    with open(merge, 'rt') as f:
        lines = f.readlines()
        y_arr = np.array([l.split()[2] for l in lines], dtype=np.float32)
        
    min_y = y_arr.min()
    v_rvh[:, 1] += min_y - v_rvh[:, 1].min()

    with open(os.path.join(OUTPUT_PATH, f'{n}.obj'),'wt') as f:
        f.writelines(lines)

        for v in v_rvh:
            x, y, z = v
            f.write(f'v {x} {y} {z} 0.000000 0.000000 1.000000\n')

In [109]:
scan_list = []

for n in names[:300]:
    scan = f'/workspace/datasets/sizekorea/rvh_test/{n}/{n}E_m.obj'

    v_scan, _, _ = load_obj(scan)
    v_scan,_   = sample_farthest_points(v_scan[None], K=6890)
    scan_list.append(v_scan)


scan_arr = torch.cat(scan_list)

rvh_list = []
hse_list = []

for n in names[:300]:
    rvh = f'/workspace/SilhouettesbasedHSE/output/sizekorea/{n}/gt.obj'
    hse = f'/workspace/SilhouettesbasedHSE/output/sizekorea/{n}/mesh_out.obj'

    v_rvh, _, _ = load_obj(rvh)
    v_hse, _, _ = load_obj(hse)

    rvh_list.append(v_rvh)
    hse_list.append(v_hse)

rvh_arr = torch.cat(rvh_list)
hse_arr = torch.cat(hse_list)

rvh_arr = rvh_arr.reshape(scan_arr.shape)
hse_arr = hse_arr.reshape(scan_arr.shape)

In [124]:
from multiprocessing.dummy import Pool 

DIR_PATH = '/workspace/SilhouettesbasedHSE/concat_sizekorea'

def _write_obj(name, scan, rvh, hse):
    lines = []
    
    rgbs = [
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]
    ]

    for s, rgb in zip([scan, rvh, hse], rgbs):
        r,g,b = rgb
        for x,y,z in s:
            l = f'v {x} {y} {z} {r} {g} {b}\n'
            lines.append(l)

    with open(name, 'wt') as f:
        f.writelines(lines)

def worker(args):
    name, v_scan, v_rvh, v_hse = args

    min_y = min(v_scan[:, 1].min(), v_rvh[:, 1].min(),  v_hse[:, 1].min())
    
    v_scan[:, 1] += min_y - v_scan[:, 1].min() 
    v_rvh[:, 1] += min_y - v_rvh[:, 1].min() 
    v_hse[:, 1] += min_y - v_hse[:, 1].min()

    v_scan = v_scan.clone()
    v_rvh = v_rvh.clone()
    v_hse = v_hse.clone()

    v_scan[:, 0] -= v_scan[:,0].max() - v_scan[:, 0].min() + 0.1
    v_hse[:, 0] += v_hse[:,0].max() - v_hse[:, 0].min() + 0.1

    scan_height = v_scan[:,1].max() - v_scan[:, 1].min()
    rvh_height = v_rvh[:,1].max() - v_rvh[:, 1].min()
    hse_height = v_hse[:,1].max() - v_hse[:, 1].min()

    v_rvh[:, 1] *= scan_height / rvh_height
    v_hse[:, 1] *= scan_height / hse_height

    path = os.path.join(DIR_PATH, f'{name}.obj')
    _write_obj(path, v_scan, v_rvh, v_hse)

args = zip(names[:300], scan_arr, rvh_arr, hse_arr)

with Pool() as pool:
    pool.map(worker, args)